# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     | Mateo Tavares  |
| **Fecha**      | 15/09/2025  |
| **Expediente** | 746004  |

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

Para poder generalizar el modelo, si utilizamos todos los datos es probable que la regresión haga overfit, y que por lo tanto no podamos extrapolar de forma precisa.

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

Sí, sería mejor, pero depende de cómo estén distribuidos nuestros datos, es el problema del overfit.

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [2]:
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import pandas as pd
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import scipy.stats as st

In [3]:
mtr=pd.read_excel('Motor Trend Car Road Tests.xlsx')

In [4]:
mtr.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [5]:
mtr=mtr.drop(columns=['model'])

In [6]:
mse=[]
for i in range(len(mtr)):
    mtrtest=mtr.iloc[[i]]
    mtrcross=mtr.drop(i)

    X_train = mtrcross.drop(columns=['mpg'])
    y_train = mtrcross['mpg']

    X_test = mtrtest.drop(columns=['mpg'])
    y_test = mtrtest['mpg']

    lr=LinearRegression()
    reg=lr.fit(X_train, y_train)

    y_pred_test = reg.predict(X_test)

    error=mean_squared_error(y_test, y_pred_test)
    mse.append(error)



In [7]:
media = np.mean(mse)
desv = np.std(mse) 
media, desv

(12.181558006901954, 17.067399871888544)

Así entonces, podríamos sacar un intervalo de confianza para ver cuando hagamos la regresión con datos completos si es que el error cuadrático entra dentro de dicho intervalo, sin embargo, podemos ver que la desviación estándar es muy alta, lo que significa que no sería muy buena idea hacer una regresión con los datos completos ya que no generalizaría, podríamos entonces probar otros modelos que no sean lineales.

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [1]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Dataset Shape: (20640, 8) (20640,)
Dataset Features: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
Dataset Target: ['MedHouseVal']


In [20]:
housing_X = pd.DataFrame(data=X, columns=housing.feature_names)
housing_X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


In [19]:
housing_Y=pd.DataFrame(data=y, columns=housing.target_names)
housing_Y.head()

,MedHouseVal
0,4.526
1,3.585
2,3.521
3,3.413
4,3.422


In [28]:
from sklearn.model_selection import KFold
x= housing_X.values  
y = housing_Y.values  
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=137)

mse1= []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    reg1 = LinearRegression()
    reg1.fit(X_train, y_train)
    

    y_pred = reg1.predict(X_test)
    
    mse_fold = mean_squared_error(y_test, y_pred)
    mse1.append(mse_fold)

In [27]:
print("MSE:", mse1)
media1=np.mean(mse1)
desv1=np.std(mse1)
print("Media:", media1)
print("Desviación estándar:", desv1)

MSE: [0.5590019245025505, 0.5531232970331563, 0.48116127963687694, 0.5742036665176024, 0.4889758378191388, 0.5283881757036983, 0.551178238028075, 0.47483961298018185, 0.5643692207196755, 0.5500253663196353]
Media: 0.532526661926059
Desviación estándar: 0.03522806775487837


Como la desviación estándar del error cuadrado es muy baja, creo que en este dataset sí podríamos utilizar todos los datos para hacer la regresión, ya que los errores no varían mucho. A diferencia del dataframe de Motor.


## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3